In [3]:
import pandas as pd
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal
from osgeo import ogr
from datetime import datetime
import re
import subprocess
from shapely.geometry import Polygon

In [7]:
now=datetime.now()
now.strftime("%Y%m%d")

'20230711'

In [10]:
g=re.search('^(.*?)_norm','384768_norm.csv')

In [12]:
g.group(1)

'384768'

In [4]:
def createid(df):        
    df['xposst'] = (df['x'] * 10000).apply('{:06.0f}'.format)
    df['yposst'] = (df['y'] * 10000).apply('{:06.0f}'.format)
    df['id'] = df['xposst']+df['yposst']
    df['id']=df['id'].astype(np.int64)
    df.drop(columns=['xposst', 'yposst'],inplace=True)

In [14]:
df=pd.read_csv("/home/lstam/Documents/daily_rasters/csv/20230630_norm_pred.csv")
#dfun=pd.read_csv("/home/lstam/Documents/daily_rasters/csv/20230630.csv")
#df.rename(columns={'xu':'x', 'yu':'y'}, inplace=True)

In [22]:
df.set_index(["y", "x"], inplace=True)

In [28]:
df

,,id,ypred0,ypred1,risk,geometry
y,x,,,,,
40.2300,19.4770,194770402300,0.938307,0.061693,1,POINT (19.47700 40.23000)
40.2248,19.4770,194770402248,0.942229,0.057771,1,POINT (19.47700 40.22480)
40.2197,19.4770,194770402197,0.938997,0.061003,1,POINT (19.47700 40.21970)
40.2300,19.4821,194821402300,0.922193,0.077807,1,POINT (19.48210 40.23000)
40.2248,19.4821,194821402248,0.944567,0.055433,1,POINT (19.48210 40.22480)
...,...,...,...,...,...,...
36.3796,28.2343,282343363796,0.171330,0.828670,5,POINT (28.23430 36.37960)
36.4415,28.2395,282395364415,0.964579,0.035421,1,POINT (28.23950 36.44150)
36.3900,28.2395,282395363900,0.911595,0.088405,1,POINT (28.23950 36.39000)


In [24]:
ds = xr.Dataset.from_dataframe(df)

In [27]:
ds.dims['y']

1337

In [5]:
createid(dfun)

In [6]:
dfun['id']

0         194770402300
1         194770402248
2         194770402197
3         194821402300
4         194821402248
              ...     
654106    282343363796
654107    282395364415
654108    282395363900
654109    282395363848
654110    282395363796
Name: id, Length: 654111, dtype: int64

In [7]:
dfm=df.merge(dfun, on='id')

In [8]:
dfm.rename(columns={'x_y':'x', 'y_y':'y'}, inplace=True)

In [9]:
dfm=dfm[['id','risk', 'x', 'y']]

In [16]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x'],df['y'],z=None, crs=4326))

In [29]:
gdf.total_bounds

array([19.477 , 34.8075, 28.2395, 41.766 ])

In [14]:
gdfm2100=gdfm.to_crs(2100)

In [16]:
gdfm2100.to_file("/home/lstam/Documents/daily_rasters/tif/20230630_pred.shp")

In [21]:
vector_fname="/home/lstam/Documents/daily_rasters/tif/20230630_pred.shp"
gg = ogr.Open(vector_fname)
lyr = gg.GetLayer(0)

In [22]:
lyr.GetName()

'20230630_pred'

In [24]:
gdfm2100.total_bounds #minx, miny, maxx, maxy

array([ 114937.22035807, 3851418.71652404,  880209.28409027,
       4627151.88774326])

y:1337, x: 1701

In [25]:
gdfm.total_bounds

array([19.47697963, 34.80754861, 28.2394718 , 41.76599828])

In [17]:
gdf.to_file("/home/lstam/Documents/daily_rasters/tif/20230630_pred_4326.shp")

In [18]:
ex=subprocess.run(['gdal_rasterize', '-a', 'risk', '-te', '19.47697963', '34.80754861', '28.2394718' , '41.76599828',\
                '-ts', '1701', '1337', "/home/lstam/Documents/daily_rasters/tif/20230630_pred_4326.shp",\
                "/home/lstam/Documents/daily_rasters/tif/20230630_pred_4326.tif"]) 

In [30]:
subprocess.run(['gdalwarp', '-cutline', '/home/lstam/Documents/newriskmodel/aktogrammh/aktogrammh.shp', '-crop_to_cutline', '-dstalpha', \
'/home/lstam/Documents/daily_rasters/tif/20230630_cells.tif', '/home/lstam/Documents/daily_rasters/tif/20230630_clip_cells.tif'])

CompletedProcess(args=['gdalwarp', '-cutline', '/home/lstam/Documents/newriskmodel/aktogrammh/aktogrammh.shp', '-crop_to_cutline', '-dstalpha', '/home/lstam/Documents/daily_rasters/tif/20230630_cells.tif', '/home/lstam/Documents/daily_rasters/tif/20230630_clip_cells.tif'], returncode=1)

gdalwarp -cutline /home/lstam/Documents/newriskmodel/aktogrammh/aktogrammh.shp -crop_to_cutline -dstalpha
/home/lstam/Documents/daily_rasters/tif/20230630_cells.tif /home/lstam/Documents/daily_rasters/tif/20230630_clip_cells.tif

In [4]:
gdf = gpd.read_file("/home/lstam/Documents/newriskmodel/aktogrammh/aktogrammh.shp")

In [5]:
gdf

,LENGTH,TYPE,geometry
0,1085.2790,KRATOS,"LINESTRING (189878.281 4409939.000, 189879.906..."
1,13590.8900,KRATOS,"LINESTRING (183230.171 4409021.501, 183258.202..."
2,3756.3480,KRATOS,"LINESTRING (183372.844 4406140.500, 183367.905..."
3,2476.1690,KRATOS,"LINESTRING (184705.687 4404287.501, 184706.203..."
4,596.3718,KRATOS,"LINESTRING (184588.328 4403728.501, 184598.202..."
...,...,...,...
5447,6108.2250,AKTH,"LINESTRING (506265.687 3858623.000, 506269.500..."
5448,6495.5300,AKTH,"LINESTRING (503385.313 3856994.001, 503379.001..."
5449,6352.2100,AKTH,"LINESTRING (510556.188 3857422.750, 510557.500..."
5450,8834.6060,AKTH,"LINESTRING (508874.813 3852129.001, 508843.093..."


In [6]:
gdf.geometry = gdf.geometry.apply(lambda x: Polygon(x.coords))

ValueError: A LinearRing must have at least 3 coordinate tuples

In [9]:
gdf['TYPE'].unique()

array(['KRATOS', 'AKTH'], dtype=object)

In [16]:
gdf.iloc[0:3].geometry

0    LINESTRING (189878.281 4409939.000, 189879.906...
1    LINESTRING (183230.171 4409021.501, 183258.202...
2    LINESTRING (183372.844 4406140.500, 183367.905...
Name: geometry, dtype: geometry

In [17]:
gdf1 = gpd.read_file("/home/lstam/Documents/newriskmodel/CNTR_BN_01M_2020_4326.shp/CNTR_BN_01M_2020_4326.shp")

In [19]:
gdf1[gdf1['CNTR_CODE']=='GR']

,EU_FLAG,EFTA_FLAG,CC_FLAG,CNTR_BN_ID,OTHR_FLAG,COAS_FLAG,POL_STAT,CNTR_BN_CO,CNTR_CODE,FID,geometry


In [22]:
gdf1['CNTR_BN_ID'].unique()

array([  522,   523,   524, ..., 67301, 67302, 67303])